# Transformando latitude e longitude em código de país padrão ISO 3166-1 alpha-2

Agradecimento ao OpenStreetMaps pelo API gratuito: https://www.openstreetmap.org/copyright

## Imports

In [8]:
import time
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm # para mostrar barra de progresso

!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


## Teste rápido do API

### Especificando o user agent para conformidade com as exigências do Nominatim
- Vide [termos de serviço do Nominatim](https://operations.osmfoundation.org/policies/nominatim/)

In [9]:
geolocator = Nominatim(user_agent="puc-ai-ml")

### Testando o API com as coordenadas do endereço da PUC Minas - Coração Eucarístico

#### Todos os dados retornados pelo API

In [10]:
location = geolocator.reverse("-19.9252563, -43.994815")

location.raw

{'place_id': 284025437,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 959560265,
 'lat': '-19.925225635433545',
 'lon': '-43.994790528155825',
 'display_name': 'Avenida 31 de Março, Dom Cabral, Regional Noroeste, Belo Horizonte, Região Geográfica Imediata de Belo Horizonte, Região Metropolitana de Belo Horizonte, Região Geográfica Intermediária de Belo Horizonte, Minas Gerais, Região Sudeste, 30535-901, Brasil',
 'address': {'road': 'Avenida 31 de Março',
  'neighbourhood': 'Dom Cabral',
  'suburb': 'Dom Cabral',
  'city_district': 'Regional Noroeste',
  'city': 'Belo Horizonte',
  'municipality': 'Região Geográfica Imediata de Belo Horizonte',
  'county': 'Região Metropolitana de Belo Horizonte',
  'state_district': 'Região Geográfica Intermediária de Belo Horizonte',
  'state': 'Minas Gerais',
  'ISO3166-2-lvl4': 'BR-MG',
  'region': 'Região Sudeste',
  'postcode': '30535-901',
  'country': 'Brasil',
  'country_co

#### Extraindo apenas o código de país alpha-2

In [4]:
location.raw.get("address", {}).get("country_code")

'br'

## Transformando as latitudes e longitudes do dataset em código de país

### Carregando o dataset
- Obs: supõe-se que o caderno exploring_data_1.ipynb já foi rodado e temos disponível o dataset em formato pickle

In [5]:
decompressed_filepath = "../datasets/10kSongs_1.pickle"
df = pd.read_pickle(decompressed_filepath)

In [14]:
# Utilizaremos um loop em vez de operações vetoriais para controlar a velocidade
# O API Nominatim só permite uma request por segundo, então iremos utilizar time.sleep
# a cada interação
country_code_series = pd.Series([], dtype=str)
for index, row in tqdm(df.iterrows(), total=len(df)):
    location = geolocator.reverse(f'{row["artist_latitude"]}, {row["artist_longitude"]}')
    # Utilizando um chaining de gets pra evitar key errors concisamente
    country_code = getattr(location, "raw", {}).get("address", {}).get("country_code")
    country_code_series.at[index] = country_code
    time.sleep(1.1)


  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
df_artist_country_code_col = pd.DataFrame(country_code_series)
df["artist_country_code"] = df_artist_country_code_col 

In [9]:
df.head(5)

In [ ]:
df.to_pickle("../datasets/10kSongs_1_with_countries.pickle")